In [1]:
%pip install --upgrade diffusers transformers scipy

  Using cached diffusers-0.3.0-py3-none-any.whl (153 kB)
  Using cached transformers-4.22.1-py3-none-any.whl (4.9 MB)
  Using cached huggingface_hub-0.9.1-py3-none-any.whl (120 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.4.0
    Uninstalling huggingface-hub-0.4.0:
      Successfully uninstalled huggingface-hub-0.4.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.21.0
    Uninstalling transformers-4.21.0:
      Successfully uninstalled transformers-4.21.0
Note: you may need to restart the kernel to use updated packages.


In [5]:
# !huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
        
Token: Traceback (most recent call last):
  File "/home/dwip.dalal/anaconda3/envs/pal/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
  File "/home/dwip.dalal/anaconda3/envs/pal/lib/python3.8/site-packages/huggingface_hub/commands/huggingface_cli.py", line 41, in main
 

In [6]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline

model_id = "CompVis/stable-diffusion-v1-4"
device = "cuda"


pipe = StableDiffusionPipeline.from_pretrained(model_id, use_auth_token=True)
pipe = pipe.to(device)

Downloading:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/342 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/71.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/522 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/335M [00:00<?, ?B/s]

{'trained_betas'} was not found in config. Values will be initialized to default values.


In [7]:
import torch
import cv2
import numpy as np
from torch import nn
from PIL import Image
from numpy import asarray

In [8]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)
model.eval()
lenet = nn.Sequential(*list(model.children())[:-2])

Using cache found in /home/dwip.dalal/.cache/torch/hub/pytorch_vision_v0.10.0


In [9]:
def score(prompt):
  ## generate image and it's feature vector
  with autocast("cuda"):
    image = pipe(prompt, guidance_scale=10)["sample"][0]  
  s_image = image
  image = cv2.resize(np.asarray(image), (224, 224))
  image = image.transpose([2, 0, 1])
  features = lenet(torch.unsqueeze(torch.Tensor(image), axis=0)).squeeze().detach().numpy()
  s_image.save("image.png")

  ### read the constant image
  img = Image.open('Andromeda_Galaxy_560mm_FL.jpg')
  numpydata = asarray(img)
  image2 = cv2.resize(np.asarray(numpydata), (224, 224))
  image2 = image2.transpose([2, 0, 1])
  features2 = lenet(torch.unsqueeze(torch.Tensor(image2), axis=0)).squeeze().detach().numpy()

  # generate score
  A = features
  B = features2

  from numpy.linalg import norm
  return(np.dot(A,B)/(norm(A)*norm(B)))


In [10]:
score('galaxy moving in space')

  0%|          | 0/51 [00:00<?, ?it/s]

0.9057495